# Technical Indicators Demo

## Download example datasets

Before getting started, let's download the example datasets if not present.

In [1]:
! ((test ! -f '../data/stock_price_hist.csv.gz' ||  test ! -f '../data/security_master.csv.gz') && \
  cd ../.. && bash download_data.sh) || echo "Dataset is already present. No need to re-download it." 

Dataset is already present. No need to re-download it.


In [2]:
from bqplot import OHLC, LinearScale, DateScale, Axis, Figure, Bars, Lines, Tooltip
from bqplot.colorschemes import CATEGORY20
import datetime
import ipywidgets as widgets
import numpy as np
import pandas as pd
import cupy as cp

In [3]:
import sys; sys.path.insert(0, '../..')

from gquant.dataframe_flow import TaskSpecSchema


task_load_csv_data = {
    TaskSpecSchema.task_id: "load_csv_data",
    TaskSpecSchema.node_type: "CsvStockLoader",
    TaskSpecSchema.conf: {"file": "../data/stock_price_hist.csv.gz"},
    TaskSpecSchema.inputs: {}
}

task_sort = {
    TaskSpecSchema.task_id: "sort",
    TaskSpecSchema.node_type: "SortNode",
    TaskSpecSchema.conf: {"keys": ['asset', 'datetime']},
    TaskSpecSchema.inputs: {"in": "load_csv_data.cudf_out"}
}

task_stock_symbol = {
    TaskSpecSchema.task_id: "stock_symbol",
    TaskSpecSchema.node_type: "StockNameLoader",
    TaskSpecSchema.conf: {"file": "../data/security_master.csv.gz"},
    TaskSpecSchema.inputs: {}
}    

In [4]:
import os
import warnings; warnings.simplefilter("ignore")

from gquant.dataframe_flow import TaskGraph

task_list = [task_load_csv_data, task_sort]
task_graph = TaskGraph(task_list)

df = task_graph.run(outputs=['sort.out'])[0]

def one_stock(df, stock_id):
    return df.query('asset==%s' % stock_id)

def slice_stock(df, year):
    beg_date = datetime.datetime.strptime(str(year)+'-01-01', '%Y-%m-%d')
    end_date = datetime.datetime.strptime(str(int(year)+1)+'-01-01', '%Y-%m-%d')
    return df.query('datetime<@end_date and datetime>=@beg_date')

indicator_lists = ['Accumulation Distribution', 'ADMI', 'Average True Range', 'Bollinger Bands',
                   'Chaikin Oscillator', 'Commodity Channel Index', 'Coppock Curve', 'Donchian Channel',
                   'Ease of Movement', 'EWA', 'Force Index', 'Keltner Channel', 'KST Oscillator', 'MA', 'MACD',
                   'Mass Index', 'Momentum', 'Money Flow Index', 'On Balance Volume', 'Parabolic SAR',
                   'Rate of Change', 'RSI', 'Stochastic Oscillator D', 'Stochastic Oscillator K', 'TRIX',
                   'True Strength Index', 'Ultimate Oscillator', 'Vortex Indicator',]

task_stocks_list = [task_stock_symbol]
task_stocks_graph = TaskGraph(task_stocks_list)
list_stocks = task_stocks_graph.run(outputs=['stock_symbol.stock_name'])[0].to_pandas().set_index('asset_name').to_dict()['asset']

main_figure_height='300px'
indicator_figure_height='200px'
figure_width = '1000px'

In [5]:
add_stock_selector = widgets.Dropdown(options=list_stocks.keys(), value=None, description="Add stock")
year_selector = widgets.IntSlider(description="All Year", continuous_update=False)
year_selectors = []

def get_figure(selected, df):
    this_stock = one_stock(df, list_stocks[selected])
    this_stock_store = [this_stock]
    stock_selector = widgets.Dropdown(options=list_stocks.keys(), value=add_stock_selector.value, description="stock")
    indicator_selector = widgets.Dropdown(options=indicator_lists, value=None, description="Indicator")
    min_year = this_stock.datetime.dt.year.min()
    max_year = this_stock.datetime.dt.year.max()
    year_selector = widgets.IntSlider(min=min_year, max=max_year, description="Year", continuous_update=False)
    year = year_selector.value
    stock = slice_stock(this_stock, year)
    sc = LinearScale()
    sc2 = LinearScale()
    dt_scale = DateScale()
    ax_x = Axis(label='Date', scale=dt_scale)
    ax_y = Axis(label='Price', scale=sc, orientation='vertical', tick_format='0.0f')
    # Construct the marks
    ohlc = OHLC(x=stock.datetime.to_array(), y=cp.asnumpy(stock[['open','high','low', 'close']].values), marker='candle', scales={'x': dt_scale, 'y': sc}, format='ohlc',
                stroke='blue', display_legend=True, labels=[selected])
    bar = Bars(x=stock.datetime.to_array(), y=stock.volume.to_array(), 
           scales={'x': dt_scale, 'y': sc2}, padding=0.2)
    def_tt = Tooltip(fields=['x', 'y'], formats=['%Y-%m-%d', '.2f'])
    bar.tooltip = def_tt
    bar.interactions = {
        'legend_hover': 'highlight_axes',
        'hover': 'tooltip', 
        'click': 'select',
    }
    sc.min = stock.close.min() - 0.3 * (stock.close.max() - stock.close.min()) 
    sc.max = stock.close.max()
    sc2.max = stock.volume.max()*4.0
    dt_scale.min = pd.Timestamp('%d-1-1' % year)
    dt_scale.max = pd.Timestamp('%d-1-1' % (year + 1))
    f = Figure(axes=[ax_x, ax_y], marks=[ohlc, bar], fig_margin={"top":0, "bottom":60, "left":60, "right":60})
    f.layout.height = main_figure_height
    f.layout.width = figure_width
    para_selectors = widgets.VBox([])
    color_id = [0]
    
    def update_graph(stock):
        with bar.hold_trait_notifications() as bc, ohlc.hold_trait_notifications() as oc:
            ohlc.y = cp.asnumpy(stock[['open','high','low', 'close']].values)
            ohlc.x = stock.datetime.to_array()
        
            bar.y = stock.volume.to_array()
            bar.x = stock.datetime.to_array()
    
            sc.min = stock.close.min() - 0.3 * (stock.close.max() - stock.close.min()) 
            sc.max = stock.close.max()
            sc2.max = stock.volume.max()*4.0
            dt_scale.min = pd.Timestamp('%d-1-1' % year_selector.value)
            dt_scale.max = pd.Timestamp('%d-1-1' % (year_selector.value + 1))
            update_range()
    
    def year_selection(*stock):
        stock = slice_stock(this_stock_store[0], year_selector.value)
        update_graph(stock)
    
    def stock_selection(*stock):
        this_stock_store[0] = one_stock(df, list_stocks[stock_selector.value])
        year_selector.min = this_stock_store[0].datetime.dt.year.min()
        year_selector.max = this_stock_store[0].datetime.dt.year.max()
        stock = slice_stock(this_stock_store[0], year_selector.value)
        ohlc.labels = [stock_selector.value]
        update_graph(stock)
        
    def update_figure_(stock, objects):
        line = objects[0]
        with line.hold_trait_notifications():
            line.y = stock['out'].to_array()
            line.x = stock.datetime.to_array()
            
    def add_new_indicator(new_fig):
        # add new figure
        # take the axis from the fig
        empty = {"top":0, "bottom":0, "left":60, "right":60}
        axis_margin = {"top":0, "bottom":60, "left":60, "right":60}
        axes_copy = multiple_figs.children[-1].axes.copy() 
        multiple_figs.children[-1].fig_margin = empty
        to_be_removed = axes_copy[0]
        axes_copy.remove(to_be_removed)
        multiple_figs.children[-1].axes = axes_copy
        new_axes = new_fig.axes.copy()
        new_fig.axes = [to_be_removed] + new_axes
        new_fig.fig_margin = axis_margin
        # add new figure
        multiple_figs.children += (new_fig,)
        
    def indicator_selection(*stock):
        if indicator_selector.value is None:
            return
        
        color_id[0] = (color_id[0] + 1) % len(CATEGORY20)
        
        def setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun):
            with out:                    
                def update_df(para_selector_widgets):
                    my_stock = this_stock_store[0]
                    parameters = get_parameters(my_stock, para_selector_widgets)
                    output = indicator_fun(*parameters)
                    stock_df = process_outputs(output, my_stock)
                    stock = slice_stock(stock_df, year_selector.value)
                    return stock
                para_selector_widgets = get_para_widgets()   
                para_selectors.children += tuple(para_selector_widgets)                    
                stock = update_df(para_selector_widgets)
                figs = create_figure(stock=stock, dt_scale=dt_scale, sc=sc, color_id=color_id, f=f, 
                                     indicator_figure_height=indicator_figure_height, figure_width=figure_width,
                                     add_new_indicator=add_new_indicator)

                def update_para(*para):
                    stock = update_df(para_selector_widgets)
                    update_figure(stock, figs)
                    
                for selector in para_selector_widgets:
                    selector.observe(update_para, 'value')
                year_selector.observe(update_para, 'value')
                stock_selector.observe(update_para, 'value')

        with out:
            update_figure = update_figure_
            if indicator_selector.value=='Accumulation Distribution':
                from viz.accumulation_distribution import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun              

            elif indicator_selector.value=='ADMI':
                from viz.admi import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun

            elif indicator_selector.value=='Average True Range':
                from viz.average_true_range import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun

            elif indicator_selector.value=='Bollinger Bands':
                from viz.bollinger_bands import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun, update_figure

            elif indicator_selector.value=='Chaikin Oscillator':
                from viz.ch_oscillator import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun

            elif indicator_selector.value=='Commodity Channel Index':
                from viz.commodity_channel_index import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun

            elif indicator_selector.value=='Coppock Curve':
                from viz.coppock_curve import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun    

            elif indicator_selector.value=='Donchian Channel':
                from viz.donchian_channel import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun      

            elif indicator_selector.value=='Ease of Movement':
                from viz.ease_of_movement import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun             

            elif indicator_selector.value=='EWA':
                from viz.ewa import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun

            elif indicator_selector.value=='Force Index':
                from viz.force_index import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun

            elif indicator_selector.value=='Keltner Channel':
                from viz.keltner_channel import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun, update_figure       

            elif indicator_selector.value=='KST Oscillator':
                from viz.kst_oscillator import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun

            elif indicator_selector.value=='MA':
                from viz.ma import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun

            elif indicator_selector.value=='MACD':           
                from viz.macd import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun, update_figure

            elif indicator_selector.value=='Mass Index':
                from viz.mass_index import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun
            elif indicator_selector.value=='Momentum':
                from viz.momentum import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun

            elif indicator_selector.value=='Money Flow Index':
                from viz.money_flow_index import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun

            elif indicator_selector.value=='On Balance Volume':
                from viz.on_balance_volume import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun

            elif indicator_selector.value=='Parabolic SAR':
                from viz.parabolic_sar import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun, update_figure

            elif indicator_selector.value=='Rate of Change':
                from viz.rate_of_change import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun

            elif indicator_selector.value=='RSI':
                from viz.rsi import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun

            elif indicator_selector.value=='Stochastic Oscillator D':
                from viz.stochastic_oscillator_d import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun

            elif indicator_selector.value=='Stochastic Oscillator K':
                from viz.stochastic_oscillator_k import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun            

            elif indicator_selector.value=='TRIX':
                from viz.trix import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun

            elif indicator_selector.value=='True Strength Index':
                from viz.true_strength_index import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun              

            elif indicator_selector.value=='Ultimate Oscillator':
                from viz.ultimate_oscillator import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun                  
                
            elif indicator_selector.value=='Vortex Indicator':
                from viz.vortex_indicator import get_para_widgets, get_parameters, process_outputs, create_figure, indicator_fun

        setup_indicator(get_para_widgets, get_parameters, process_outputs, create_figure, update_figure, indicator_fun)

        indicator_selector.value=None

    year_selector.observe(year_selection, 'value')
    stock_selector.observe(stock_selection, 'value')
    indicator_selector.observe(indicator_selection, 'value')
    multiple_figs = widgets.VBox([f])
    return multiple_figs, year_selector, stock_selector, indicator_selector, para_selectors

def stock_selection(*stock):
    if add_stock_selector.value is None:
        return
    f, year_selector, stock_selector, indicator_selector, para_selectors = get_figure(add_stock_selector.value, df)
    vbox = w.children[1]
    vbox.children += (widgets.HBox([f, widgets.VBox([year_selector, stock_selector, indicator_selector, para_selectors])]),) 
    year_selectors.append(year_selector)
    update_range()
    add_stock_selector.value = None

def update_range():
    min_vals = []
    max_vals = []
    for i in year_selectors:
        min_vals.append(i.min)
        max_vals.append(i.max)
    minV = max(min_vals)
    maxV = min(max_vals)
    if minV<=maxV:
        year_selector.disabled = False
        year_selector.max = maxV   
        year_selector.min = minV
    else:
        year_selector.disabled = True
    

out = widgets.Output(layout={'border': '1px solid black'})

def update_all_ranges(*arg):
    for i in year_selectors:
        i.value = year_selector.value
    
add_stock_selector.observe(stock_selection, 'value')
year_selector.observe(update_all_ranges, 'value')
selectors = widgets.HBox([add_stock_selector, year_selector])
w = widgets.VBox([selectors, widgets.VBox([])])
w

In [6]:
out

Output(layout=Layout(border='1px solid black'))